In [2]:
import sys,os,datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
from __future__ import print_function

In [3]:
print(np.__version__);
print(pd.__version__);

1.12.1
0.19.2


In [4]:
import  cv2 as cv
cv.__version__

'3.2.0'

In [5]:
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential,save_model,load_model
from keras.layers import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D
from keras.layers import Activation, Dropout, Flatten, Dense
import keras.optimizers

Using TensorFlow backend.


In [6]:
teDir    = '../Data/test-tif-v2'
teDirTIF = teDir
teDirJPG = '../Data/test-jpg'

In [15]:
nameList  = os.listdir(teDirTIF);
print(len(nameList),nameList[:6]);

61191 ['test_28790.tif', 'test_28791.tif', 'test_28792.tif', 'test_28793.tif', 'test_28794.tif', 'test_28795.tif']


In [18]:
def formFH (nf, bins, printOK=False) :
    nx = None
    try : 
        ni = cv.imread(nf,-1); 
        if (ni is not None) :
            if not ((ni.shape[2]==3) or (ni.shape[2]==4)) and printOK : print('----- error ---- shape:',ni.shape,nf)
            if (ni.shape[2]==3) :
                r,g,b = ni[:,:,0],ni[:,:,1],ni[:,:,2]
                rh = np.divide(np.histogram(r,bins=bins,density=False)[0],(0.0+r.size))
                gh = np.divide(np.histogram(g,bins=bins,density=False)[0],(0.0+g.size))
                bh = np.divide(np.histogram(b,bins=bins,density=False)[0],(0.0+b.size))
                nx = np.hstack((rh,gh,bh)); 
            if (ni.shape[2]==4) :
                r,g,b,n = ni[:,:,2],ni[:,:,1],ni[:,:,1],ni[:,:,3]
                d  = np.divide((r-n),(r+n+0.01))
                rh = np.divide(np.histogram(r,bins=bins,density=False)[0],(0.0+r.size))
                gh = np.divide(np.histogram(g,bins=bins,density=False)[0],(0.0+g.size))
                bh = np.divide(np.histogram(b,bins=bins,density=False)[0],(0.0+b.size))
                nh = np.divide(np.histogram(n,bins=bins,density=False)[0],(0.0+n.size))
                dh = np.divide(np.histogram(d,bins=bins,density=False)[0],(0.0+d.size))
                nx = np.hstack((rh,gh,bh,nh,dh)); 
    except e :
        print(nf,e); nx = None;
    
    if (nx is None) and printOK : print('------ None:',nf)
        
    return(nx)


In [19]:
trX, trY, i, size = [], [], 0, 70000
step=10; bins=[x for x in range(0,255+step,step)] # RGB -> 0..255
step=300; bins=[x for x in range(0,65535+step+1,step)] # BGRN -> 0..65535
print(datetime.datetime.now(),len(nameList),size)
for nn in nameList[0:size] :
    #nf = os.path.join(trDirJPG,nn+".jpg");
    #nf = os.path.join(trDirTIF,nn+".tif");
    nf = os.path.join(teDirTIF,nn);
    nx = formFH(nf,bins)
    if (nx is not None) :
        nx=nx.reshape(nx.shape+(1,))
        trX.append(nx)
        trY.append(nn)
    i += 1
    if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)
    #print(nn.shape)
    
print(datetime.datetime.now())

len(trX)
trX = np.array(trX);
trY = np.array(trY);
print(len(nameList),trX.shape,trY.shape)

2017-05-13 00:18:45.531231 61191 70000
2017-05-13 00:23:37.012809 	 5000 	 test_15649.tif
2017-05-13 00:28:17.424138 	 10000 	 test_32423.tif
2017-05-13 00:33:04.976499 	 15000 	 test_36264.tif
2017-05-13 00:37:43.444670 	 20000 	 test_10894.tif
2017-05-13 00:42:28.624183 	 25000 	 file_9502.tif
2017-05-13 00:47:04.120772 	 30000 	 file_14128.tif
2017-05-13 00:51:34.572873 	 35000 	 file_17389.tif
2017-05-13 00:56:03.436713 	 40000 	 test_19356.tif
2017-05-13 01:00:31.464483 	 45000 	 test_10134.tif
2017-05-13 01:04:58.941542 	 50000 	 test_24243.tif
2017-05-13 01:09:46.624081 	 55000 	 test_7049.tif
2017-05-13 01:14:25.789511 	 60000 	 file_10482.tif
2017-05-13 01:15:29.610939
61191 (61191, 1095, 1) (61191,)


In [24]:
trY=[os.path.splitext(yy)[0] for yy in trY.tolist()]

In [27]:
np.save('../Data-Keras/test-basin02-01-X-tif-v2.npy',trX)
np.save('../Data-Keras/test-basin02-01-Y-tif-v2.npy',trY)

In [7]:
trX = np.load('../Data-Keras/test-basin02-01-X-tif-v2.npy')
trY = np.load('../Data-Keras/test-basin02-01-Y-tif-v2.npy')
print(trX.shape,trY.shape)

(61191, 1095, 1) (61191,)


In [8]:
trL = np.load('../Data-Keras/train-basin02-01-L-tif-v2.npy')
trM = np.load('../Data-Keras/train-basin02-01-M-tif-v2.npy')
print(trL,'\n',trM)

['haze' 'primary' 'agriculture' 'clear' 'water' 'habitation' 'road'
 'cultivation' 'slash_burn' 'cloudy' 'partly_cloudy' 'conventional_mine'
 'bare_ground' 'artisinal_mine' 'blooming' 'selective_logging' 'blow_down'] 
 [ 0.49  0.5   0.52  0.49  0.54  0.5   0.5   0.47  0.7   0.6   0.6   0.77
  0.59  0.58  0.65  0.78  0.86]


In [9]:
import keras.backend as K
def fbeta_pred(y_true, y_pred, beta=2.0, OK1=0.2, eps=0.000001, printOK=False):
    beta2 = beta*beta
    yy_true = K.round(y_true)
    yy_pred = K.round(y_pred+(0.5-OK1))
    tp, tp_fp, fn = K.sum((yy_pred*yy_true)), K.sum(yy_true), K.sum((K.abs(yy_pred*(yy_true-1.0))))
    precision, recall = tp/(tp_fp+eps), tp/(tp+fn+eps) 
    if printOK :
        print('ten true ',K.get_value(yy_true))
        #print('ten pred ',y_pred)
        print('ten roun ',K.get_value(yy_pred))
        print(' pre=',K.get_value(precision),' recall=',K.get_value(recall),' tp=',
              K.get_value(tp),' fn=',K.get_value(fn),' tp+fp=',K.get_value(tp_fp))
    return((1+beta2)*(precision*recall)/(beta2*precision+recall+eps))

In [10]:
#
#
#

ss = trX.shape[1]
rr = trL.shape[0]

model = Sequential()
#model.add(Dense(ss*10, input_shape=(ss,1))) # ?????
model.add(Dense(64, input_shape=(ss,1))) # ?????
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(32768))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(rr,activation='sigmoid'))
#model.add(Dropout(0.5))
#model.add(Activation('sigmoid'))
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', #sgd, #"adam", #'rmsprop',
              metrics=[fbeta_pred]) #['accuracy',fbeta_pred]) #['accuracy'])

In [11]:
model.load_weights('../Data-Keras/basin02-loop-weights.h5') ## verify load weights from v1 version

In [12]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 1095, 64)      128         dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 1095, 64)      0           dense_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 1095, 64)      0           activation_1[0][0]               
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 70080)         0           dropout_1[0][0]                  
___________________________________________________________________________________________

In [15]:
trP = model.predict(trX, batch_size=512); 
trP.shape

(61191, 17)

In [47]:
weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
land_labels = ['primary', 'agriculture', 'water', 'cultivation', 'habitation' ]
#rare_labels = [l for l in label_list if labels_df[label_list].sum()[l] < 2000]
wr = [];
for i in weather_labels :
    wr.append(trL.tolist().index(i))
trM[np.array(wr)] = 2.0
print(wr,'\n',trL.tolist(),'\n',trM.tolist())

[3, 10, 0, 9] 
 ['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down'] 
 [2.0, 0.5, 0.52, 2.0, 0.54, 0.5, 0.5, 0.47, 0.7, 2.0, 2.0, 0.77, 0.59, 0.58, 0.65, 0.78, 0.86]


In [84]:
#trP = model.predict(trX, batch_size=512); trP=K.get_value(trP)
res = []

for i in range(trP.shape[0]) :
    trPP = [weather_labels[trP[i,wr].argmax()]] + trL[(trP[i,:]>trM)].tolist();
    pp   = ' '.join(trPP)
    ##if (pp=="") : print(trY[i])
    res.append([trY[i],pp])
res[4:11]
res.sort(cmp=lambda x,y: cmp(int(x[0].partition('_')[2]),int(y[0].partition('_')[2])) if (x[0].partition('_')[0]==y[0].partition('_')[0]) else cmp(y[0].partition('_')[0],x[0].partition('_')[0]))
         

In [87]:
res[-6:]

[['file_20516', 'clear primary'],
 ['file_20517', 'clear primary'],
 ['file_20518', 'haze primary agriculture'],
 ['file_20519', 'clear primary water'],
 ['file_20520', 'partly_cloudy primary'],
 ['file_20521', 'clear primary agriculture']]

In [88]:
print(trL.tolist())
print(trM.tolist())
np.round(trP[4:11,:])

['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down']
[2.0, 0.5, 0.52, 2.0, 0.54, 0.5, 0.5, 0.47, 0.7, 2.0, 2.0, 0.77, 0.59, 0.58, 0.65, 0.78, 0.86]


array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.]], dtype=float32)

In [89]:
rrr=pd.DataFrame(res,columns=['image_name','tags']); rrr.head(); 
suffixDT = (datetime.datetime.now()).strftime('%Y-%m-%d-%H-%M-%S'); print(suffixDT)
rrr.to_csv('../Result/vss'+suffixDT+'.csv',index=False);

2017-05-13-03-38-54
